In [1]:
import pandas as pd
import numpy as np

# Load CSV
data = pd.read_csv("/content/drive/MyDrive/Proj 2 /Supplement_Sales_Weekly_Expanded.csv")

# Rename columns to standard names
data.rename(columns={
    'Date': 'date',
    'Product Name': 'product_id',
    'Units Sold': 'quantity_sold',
    'Price': 'price',
    'Discount': 'discount'
}, inplace=True)

# Parse date column
data['date'] = pd.to_datetime(data['date'])

# Extract seasonality features
data['month'] = data['date'].dt.month
data['week'] = data['date'].dt.isocalendar().week
data['day_of_week'] = data['date'].dt.dayofweek

# Fill missing values
data['quantity_sold'] = data['quantity_sold'].fillna(0)
data['price'] = data['price'].fillna(method='ffill')
data['discount'] = data['discount'].fillna(0)

# Sort by product and date
data = data.sort_values(by=['product_id', 'date'])

print("Preprocessed data preview:")
print(data.head())


Preprocessed data preview:
         date   product_id Category  quantity_sold  price  Revenue  discount  \
10 2020-01-06  Ashwagandha   Herbal            181  15.49  2803.69      0.14   
26 2020-01-13  Ashwagandha   Herbal            133  54.35  7228.55      0.16   
42 2020-01-20  Ashwagandha   Herbal            156  23.49  3664.44      0.21   
58 2020-01-27  Ashwagandha   Herbal            174  30.94  5383.56      0.07   
74 2020-02-03  Ashwagandha   Herbal            135  19.21  2593.35      0.22   

    Units Returned Location Platform  month  week  day_of_week  
10               1      USA   Amazon      1     2            0  
26               1       UK  Walmart      1     3            0  
42               1   Canada    iHerb      1     4            0  
58               2   Canada   Amazon      1     5            0  
74               2       UK   Amazon      2     6            0  


/tmp/ipython-input-1503511484.py:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['price'] = data['price'].fillna(method='ffill')


In [2]:
from sklearn.linear_model import LinearRegression

price_recommendations = []

for product in data['product_id'].unique():
    df = data[data['product_id'] == product]

    # Features: price, discount, seasonal info
    X = df[['price', 'discount', 'month', 'week', 'day_of_week']].values
    y = df['quantity_sold'].values

    # Fit linear regression
    model = LinearRegression()
    model.fit(X, y)

    # Compute elasticity at mean price (ignoring seasonality for elasticity)
    mean_price = df['price'].mean()
    mean_qty = df['quantity_sold'].mean()
    elasticity = model.coef_[0] * (mean_price / mean_qty)

    # Price recommendation
    recommended_price = mean_price
    if abs(elasticity) > 1:
        recommended_price *= 0.95  # reduce price by 5% if elastic
    else:
        recommended_price *= 1.05  # increase price by 5% if inelastic

    price_recommendations.append({
        'product_id': product,
        'elasticity': elasticity,
        'current_price': mean_price,
        'recommended_price': recommended_price
    })

price_df = pd.DataFrame(price_recommendations)
print(price_df)


            product_id  elasticity  current_price  recommended_price
0          Ashwagandha    0.005773      33.920000          35.616000
1                 BCAA    0.017155      35.601715          37.381801
2               Biotin    0.011834      35.732080          37.518684
3    Collagen Peptides    0.001077      35.067701          36.821086
4             Creatine   -0.013428      34.819161          36.560119
5   Electrolyte Powder    0.004156      34.363139          36.081296
6             Fish Oil    0.009286      35.059015          36.811965
7    Green Tea Extract   -0.007267      35.391022          37.160573
8      Iron Supplement    0.011793      34.692774          36.427412
9            Magnesium   -0.010590      33.050292          34.702807
10           Melatonin    0.007084      33.899927          35.594923
11        Multivitamin   -0.000820      33.801934          35.492031
12         Pre-Workout   -0.000304      35.779234          37.568195
13           Vitamin C    0.006761

In [3]:
import pandas as pd

# Merge with historical average quantity sold
avg_qty = data.groupby('product_id')['quantity_sold'].mean().reset_index()
avg_qty.rename(columns={'quantity_sold': 'avg_quantity_sold'}, inplace=True)

revenue_df = pd.merge(price_df, avg_qty, on='product_id')

# Calculate projected revenue
revenue_df['current_revenue'] = revenue_df['current_price'] * revenue_df['avg_quantity_sold']
revenue_df['projected_revenue'] = revenue_df['recommended_price'] * revenue_df['avg_quantity_sold']
revenue_df['revenue_change'] = revenue_df['projected_revenue'] - revenue_df['current_revenue']
revenue_df['revenue_change_pct'] = (revenue_df['revenue_change'] / revenue_df['current_revenue']) * 100

# Sort by potential revenue gain
revenue_df = revenue_df.sort_values(by='revenue_change_pct', ascending=False)

print(revenue_df[['product_id', 'current_revenue', 'projected_revenue', 'revenue_change', 'revenue_change_pct']])


            product_id  current_revenue  projected_revenue  revenue_change  \
3    Collagen Peptides      5228.926938        5490.373285      261.446347   
2               Biotin      5416.279163        5687.093122      270.813958   
15                Zinc      5414.542362        5685.269481      270.727118   
14        Whey Protein      5186.163026        5445.471178      259.308151   
1                 BCAA      5330.772171        5597.310779      266.538609   
0          Ashwagandha      5126.129051        5382.435504      256.306453   
8      Iron Supplement      5215.817959        5476.608857      260.790898   
5   Electrolyte Powder      5150.081351        5407.585418      257.504068   
4             Creatine      5240.156591        5502.164420      262.007830   
13           Vitamin C      5184.343780        5443.560969      259.217189   
10           Melatonin      5093.031005        5347.682555      254.651550   
6             Fish Oil      5287.641527        5552.023603      

In [45]:
import plotly.express as px

# 1. Elasticity vs Product
fig1 = px.bar(price_df, x='product_id', y='elasticity',
              title='Product Elasticity', text='elasticity')
fig1.update_layout(xaxis_tickangle=-45)
fig1.show()


# 2. Projected Revenue Change
fig3 = px.bar(revenue_df, x='product_id', y='revenue_change',
              color='revenue_change_pct', title='Projected Revenue Change per Product',
              text='revenue_change', color_continuous_scale='Viridis')
fig3.update_layout(xaxis_tickangle=-45)
fig3.show()
# Only run if 'Category' column exists
if 'Category' in data.columns:
    # Use renamed column 'product_id' instead of 'Product Name'
    heat_data = data.groupby(['product_id','Category'])['quantity_sold'].mean().unstack(fill_value=0)

    import plotly.express as px
    fig4 = px.imshow(heat_data, text_auto=True, aspect="auto",
                     title='Average Sales Heatmap by Product & Category')
    fig4.show()



In [46]:
import plotly.express as px

# Get top 5 products by revenue change
top5 = revenue_df.sort_values(by='revenue_change', ascending=False).head(5)

fig = px.bar(
    top5,
    x='revenue_change',
    y='product_id',
    orientation='h',
    text='revenue_change',
    color='revenue_change_pct',
    color_continuous_scale='Viridis',
    title='Top 5 Products by Projected Revenue Increase',
    labels={'revenue_change':'Revenue Change', 'product_id':'Product'}
)

fig.update_layout(
    title_font_size=22,
    xaxis_tickformat='$,.0f',
    yaxis=dict(categoryorder='total ascending'),
    plot_bgcolor='white',
    yaxis_title='',
    xaxis_title='Projected Revenue Change',
)

fig.show()


In [7]:
import plotly.express as px

fig = px.scatter(
    revenue_df,
    x='elasticity',
    y='revenue_change',
    size='avg_quantity_sold',
    color='revenue_change_pct',
    hover_name='product_id',
    hover_data={
        'current_price': True,
        'recommended_price': True,
        'revenue_change': True,
        'elasticity': ':.3f'
    },
    color_continuous_scale='Viridis',
    title='Revenue Impact vs Price Elasticity per Product',
    labels={
        'elasticity': 'Price Elasticity',
        'revenue_change': 'Projected Revenue Change',
        'avg_quantity_sold': 'Avg Units Sold',
        'revenue_change_pct': 'Revenue Change (%)'
    },
    size_max=40,
    text='short_name' # Add this line to display product names
)

fig.update_traces(textposition='top center') # Adjust text position
fig.update_layout(
    plot_bgcolor='white',
    title_font_size=22,
    xaxis=dict(gridcolor='lightgray'),
    yaxis=dict(gridcolor='lightgray'),
)

fig.show()